In [1]:
import torch
import transformers
import sys,logging
logging.root.handlers = []
logging.basicConfig(level="INFO", format = '%(asctime)s:%(levelname)s: %(message)s' ,stream = sys.stdout)
logger = logging.getLogger(__name__)
logger.info('hello')

2020-10-22 17:18:07,664:INFO: hello


In [5]:
# Log

In [2]:
def check_memory():
    logger.info('GPU memory: %.1f' % (torch.cuda.memory_allocated() // 1024 ** 2))

In [3]:
device = torch.device('cuda')
torch.cuda.empty_cache()
check_memory()

2020-10-22 17:18:14,725:INFO: GPU memory: 0.0


In [9]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
logger.info('moving model to GPU')
gpu_model = model.to(device)
check_memory()

2020-10-19 11:46:27,983:INFO: Lock 139878498884944 acquired on /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
2020-10-19 11:46:27,985:INFO: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /.cache/torch/transformers/tmpvuzuom97



2020-10-19 11:46:28,438:INFO: storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2020-10-19 11:46:28,440:INFO: creating metadata file for /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2020-10-19 11:46:28,441:INFO: Lock 139878498884944 released on /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
2020-10-19 11:46:28,442:INFO: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca07


2020-10-19 11:46:40,673:INFO: storing https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin in cache at /.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2020-10-19 11:46:40,674:INFO: creating metadata file for /.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2020-10-19 11:46:40,675:INFO: Lock 139878323310416 released on /.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
2020-10-19 11:46:40,676:INFO: loading weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin from cache at /.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2020-

In [10]:
#Run classification for a batch of 64 tensors
def run_bert(x):
    check_memory()
    logger.info('moving tensors to GPU')
    x = x.to(device)
    check_memory()
    logger.info('Running bert forward on x')
    yhat = gpu_model(x)
    check_memory()
    logger.info(f'yhat[0].requires_grad = {yhat[0].requires_grad} . Detaching yhat')
    yhat = yhat[0].detach()
    logger.info(f'x shape = {x.shape}, yhat.shape = {yhat.shape}')
    check_memory()

In [11]:
# BERT 512 tokens

In [12]:
x = torch.randint(low =1000, high = 30000 , size = (1,512))
run_bert(x)

2020-10-19 11:47:16,402:INFO: GPU memory: 418.0
2020-10-19 11:47:16,403:INFO: moving tensors to GPU
2020-10-19 11:47:16,420:INFO: GPU memory: 418.0
2020-10-19 11:47:16,421:INFO: Running bert forward on x
2020-10-19 11:47:16,796:INFO: GPU memory: 1013.0
2020-10-19 11:47:16,797:INFO: yhat[0].requires_grad = True . Detaching yhat
2020-10-19 11:47:16,799:INFO: x shape = torch.Size([1, 512]), yhat.shape = torch.Size([1, 2])
2020-10-19 11:47:16,800:INFO: GPU memory: 418.0


In [13]:
## eval mode

In [14]:
gpu_model.eval()
run_bert(x)

2020-10-19 11:47:37,874:INFO: GPU memory: 418.0
2020-10-19 11:47:37,876:INFO: moving tensors to GPU
2020-10-19 11:47:37,892:INFO: GPU memory: 418.0
2020-10-19 11:47:37,892:INFO: Running bert forward on x
2020-10-19 11:47:37,909:INFO: GPU memory: 1013.0
2020-10-19 11:47:37,909:INFO: yhat[0].requires_grad = True . Detaching yhat
2020-10-19 11:47:37,911:INFO: x shape = torch.Size([1, 512]), yhat.shape = torch.Size([1, 2])
2020-10-19 11:47:37,912:INFO: GPU memory: 418.0


In [15]:
### With no grad

In [16]:
x.requires_grad=False
run_bert(x)

2020-10-19 11:47:55,452:INFO: GPU memory: 418.0
2020-10-19 11:47:55,454:INFO: moving tensors to GPU
2020-10-19 11:47:55,477:INFO: GPU memory: 418.0
2020-10-19 11:47:55,477:INFO: Running bert forward on x
2020-10-19 11:47:55,492:INFO: GPU memory: 1013.0
2020-10-19 11:47:55,493:INFO: yhat[0].requires_grad = True . Detaching yhat
2020-10-19 11:47:55,494:INFO: x shape = torch.Size([1, 512]), yhat.shape = torch.Size([1, 2])
2020-10-19 11:47:55,495:INFO: GPU memory: 418.0


In [17]:
# max len 256

In [19]:
x = torch.randint(low =1000, high = 30000 , size = (1,256))
run_bert(x)

2020-10-19 11:51:58,340:INFO: GPU memory: 418.0
2020-10-19 11:51:58,341:INFO: moving tensors to GPU
2020-10-19 11:51:58,372:INFO: GPU memory: 418.0
2020-10-19 11:51:58,373:INFO: Running bert forward on x
2020-10-19 11:51:58,404:INFO: GPU memory: 636.0
2020-10-19 11:51:58,405:INFO: yhat[0].requires_grad = True . Detaching yhat
2020-10-19 11:51:58,406:INFO: x shape = torch.Size([1, 256]), yhat.shape = torch.Size([1, 2])
2020-10-19 11:51:58,407:INFO: GPU memory: 418.0


In [20]:
# Max len 128

In [21]:
x = torch.randint(low =1000, high = 30000 , size = (1,128))
run_bert(x)

2020-10-19 11:52:30,793:INFO: GPU memory: 418.0
2020-10-19 11:52:30,794:INFO: moving tensors to GPU
2020-10-19 11:52:30,811:INFO: GPU memory: 418.0
2020-10-19 11:52:30,812:INFO: Running bert forward on x
2020-10-19 11:52:30,828:INFO: GPU memory: 518.0
2020-10-19 11:52:30,829:INFO: yhat[0].requires_grad = True . Detaching yhat
2020-10-19 11:52:30,830:INFO: x shape = torch.Size([1, 128]), yhat.shape = torch.Size([1, 2])
2020-10-19 11:52:30,831:INFO: GPU memory: 418.0


In [22]:
# Checkin max batch size

In [23]:
#Run classification for a batch of 64 tensors
def run_max_batch_analysis(gpu_model,  nograd = False, seq_len = 512):
    def run_bert(x):
        check_memory()
        x = x.to(device)
        yhat = gpu_model(x)
        logger.info(f'x shape = {x.shape}, yhat.shape = {yhat[0].shape}')
        check_memory()
        
    def run_bert_nograd(x):
        with torch.no_grad():
            check_memory()
            x = x.to(device)
            yhat = gpu_model(x)
            logger.info(f'x shape = {x.shape}, yhat.shape = {yhat[0].shape}')
            check_memory()


    for batch in (4,8,16,32,64,128,256,512):
        try:
            x = torch.randint(low =1000, high = 30000 , size = (batch,seq_len))
            run_bert_nograd(x) if nograd else run_bert(x)
            logger.info(f'batch size {batch} successful.')
        except Exception as e:
            print(f'exception {type(e)} : {e}')
            break

In [24]:
## Bert base model

In [25]:
del gpu_model
torch.cuda.empty_cache()
check_memory()

2020-10-19 11:53:54,140:INFO: GPU memory: 418.0


In [26]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
gpu_model = model.to(device)
check_memory()

2020-10-19 11:54:03,919:INFO: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2020-10-19 11:54:03,921:INFO: Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2020-10-19 11:54:04,077:INFO: loading weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin from cache at /.cache/torch/transformers/f2ee78bdd635

In [27]:
run_max_batch_analysis(gpu_model)

2020-10-19 11:54:13,735:INFO: GPU memory: 837.0
2020-10-19 11:54:13,778:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:54:13,779:INFO: GPU memory: 3154.0
2020-10-19 11:54:13,780:INFO: batch size 4 successful.
2020-10-19 11:54:13,782:INFO: GPU memory: 837.0
2020-10-19 11:54:13,819:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:54:13,819:INFO: GPU memory: 5470.0
2020-10-19 11:54:13,820:INFO: batch size 8 successful.
2020-10-19 11:54:13,822:INFO: GPU memory: 837.0
2020-10-19 11:54:13,919:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:54:13,920:INFO: GPU memory: 10103.0
2020-10-19 11:54:13,921:INFO: batch size 16 successful.
2020-10-19 11:54:13,922:INFO: GPU memory: 837.0
2020-10-19 11:54:14,120:INFO: x shape = torch.Size([32, 512]), yhat.shape = torch.Size([32, 2])
2020-10-19 11:54:14,122:INFO: GPU memory: 19368.0
2020-10-19 11:54:14,123:INFO: batch size 32 successful.
2020-10-

In [28]:
### seq len 128

In [29]:
run_max_batch_analysis(gpu_model,False,128)

2020-10-19 11:54:53,871:INFO: GPU memory: 837.0
2020-10-19 11:54:53,902:INFO: x shape = torch.Size([4, 128]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:54:53,903:INFO: GPU memory: 1218.0
2020-10-19 11:54:53,904:INFO: batch size 4 successful.
2020-10-19 11:54:53,905:INFO: GPU memory: 837.0
2020-10-19 11:54:53,918:INFO: x shape = torch.Size([8, 128]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:54:53,919:INFO: GPU memory: 1564.0
2020-10-19 11:54:53,920:INFO: batch size 8 successful.
2020-10-19 11:54:53,921:INFO: GPU memory: 837.0
2020-10-19 11:54:53,941:INFO: x shape = torch.Size([16, 128]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:54:53,942:INFO: GPU memory: 2290.0
2020-10-19 11:54:53,943:INFO: batch size 16 successful.
2020-10-19 11:54:53,945:INFO: GPU memory: 837.0
2020-10-19 11:54:53,986:INFO: x shape = torch.Size([32, 128]), yhat.shape = torch.Size([32, 2])
2020-10-19 11:54:53,987:INFO: GPU memory: 3742.0
2020-10-19 11:54:53,988:INFO: batch size 32 successful.
2020-10-19

For lesser sequence length, the memory usage decreases with same ratio for all batch sizes. For example,

Batch size 4 : memory consumed in forward pass

512 seq length = 3610-418 = 3192

128 seq length = 1009-418 = 591

3192/591 = 5.4

512/128 = 4

Batch size 8 : memory consumed in forward pass

512 seq length = 6803-418 = 6385

128 seq length = 1582 -418 = 1164

6384/1187 = 5.48

512/128 = 4


In [31]:
### No grad

In [32]:
run_max_batch_analysis(gpu_model, nograd = True)

2020-10-19 11:55:38,478:INFO: GPU memory: 837.0
2020-10-19 11:55:38,504:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:55:38,505:INFO: GPU memory: 837.0
2020-10-19 11:55:38,506:INFO: batch size 4 successful.
2020-10-19 11:55:38,508:INFO: GPU memory: 837.0
2020-10-19 11:55:38,560:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:55:38,561:INFO: GPU memory: 837.0
2020-10-19 11:55:38,562:INFO: batch size 8 successful.
2020-10-19 11:55:38,563:INFO: GPU memory: 837.0
2020-10-19 11:55:38,659:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:55:38,660:INFO: GPU memory: 837.0
2020-10-19 11:55:38,661:INFO: batch size 16 successful.
2020-10-19 11:55:38,662:INFO: GPU memory: 837.0
2020-10-19 11:55:38,844:INFO: x shape = torch.Size([32, 512]), yhat.shape = torch.Size([32, 2])
2020-10-19 11:55:38,845:INFO: GPU memory: 837.0
2020-10-19 11:55:38,846:INFO: batch size 32 successful.
2020-10-19 11:

In [33]:
# Bert large

In [34]:
Bert Large

SyntaxError: invalid syntax (<ipython-input-34-2658f35f5b4e>, line 1)

In [35]:
del gpu_model
torch.cuda.empty_cache()
check_memory()

2020-10-19 11:56:12,991:INFO: GPU memory: 837.0


In [36]:
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')
gpu_model = model.to(device)
check_memory()

2020-10-19 11:56:23,050:INFO: Lock 139878315749008 acquired on /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8.lock
2020-10-19 11:56:23,053:INFO: https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json not found in cache or force_download set to True, downloading to /.cache/torch/transformers/tmppp97817j



2020-10-19 11:56:23,531:INFO: storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json in cache at /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
2020-10-19 11:56:23,532:INFO: creating metadata file for /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
2020-10-19 11:56:23,533:INFO: Lock 139878315749008 released on /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8.lock
2020-10-19 11:56:23,534:INFO: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726


2020-10-19 11:57:00,835:INFO: storing https://cdn.huggingface.co/bert-large-uncased-pytorch_model.bin in cache at /.cache/torch/transformers/73e65a4648c1a5eab31ecea94e04a92a7168cd7089d588b68e5bc057aff40421.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6
2020-10-19 11:57:00,835:INFO: creating metadata file for /.cache/torch/transformers/73e65a4648c1a5eab31ecea94e04a92a7168cd7089d588b68e5bc057aff40421.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6
2020-10-19 11:57:00,836:INFO: Lock 139878315351888 released on /.cache/torch/transformers/73e65a4648c1a5eab31ecea94e04a92a7168cd7089d588b68e5bc057aff40421.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6.lock
2020-10-19 11:57:00,837:INFO: loading weights file https://cdn.huggingface.co/bert-large-uncased-pytorch_model.bin from cache at /.cache/torch/transformers/73e65a4648c1a5eab31ecea94e04a92a7168cd7089d588b68e5bc057aff40421.4d5343a4b979c4beeaadef17a0453d1bb183dd9b084f58b84c7cc781df343ae6
202

In [37]:
run_max_batch_analysis(gpu_model)

2020-10-19 11:57:09,725:INFO: GPU memory: 1697.0
2020-10-19 11:57:09,856:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:57:09,856:INFO: GPU memory: 7858.0
2020-10-19 11:57:09,858:INFO: batch size 4 successful.
2020-10-19 11:57:09,859:INFO: GPU memory: 1697.0
2020-10-19 11:57:09,922:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:57:09,922:INFO: GPU memory: 14019.0
2020-10-19 11:57:09,924:INFO: batch size 8 successful.
2020-10-19 11:57:09,925:INFO: GPU memory: 1697.0
2020-10-19 11:57:10,222:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:57:10,223:INFO: GPU memory: 26341.0
2020-10-19 11:57:10,224:INFO: batch size 16 successful.
2020-10-19 11:57:10,226:INFO: GPU memory: 1697.0
exception <class 'RuntimeError'> : CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 47.46 GiB total capacity; 46.10 GiB already allocated; 417.00 MiB free; 46.14 GiB reserved in total by PyTorch)


In [38]:
### 128 seq len

In [39]:
run_max_batch_analysis(gpu_model,False,128)

2020-10-19 11:57:11,780:INFO: GPU memory: 1697.0
2020-10-19 11:57:11,881:INFO: x shape = torch.Size([4, 128]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:57:11,881:INFO: GPU memory: 2662.0
2020-10-19 11:57:11,883:INFO: batch size 4 successful.
2020-10-19 11:57:11,884:INFO: GPU memory: 1697.0
2020-10-19 11:57:11,912:INFO: x shape = torch.Size([8, 128]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:57:11,913:INFO: GPU memory: 3626.0
2020-10-19 11:57:11,914:INFO: batch size 8 successful.
2020-10-19 11:57:11,916:INFO: GPU memory: 1697.0
2020-10-19 11:57:11,974:INFO: x shape = torch.Size([16, 128]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:57:11,975:INFO: GPU memory: 5554.0
2020-10-19 11:57:11,976:INFO: batch size 16 successful.
2020-10-19 11:57:11,977:INFO: GPU memory: 1697.0
2020-10-19 11:57:12,097:INFO: x shape = torch.Size([32, 128]), yhat.shape = torch.Size([32, 2])
2020-10-19 11:57:12,098:INFO: GPU memory: 9411.0
2020-10-19 11:57:12,100:INFO: batch size 32 successful.
2020-1

In [40]:
### No grad

In [41]:
run_max_batch_analysis(gpu_model,nograd = True)

2020-10-19 11:57:24,786:INFO: GPU memory: 1697.0
2020-10-19 11:57:24,813:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 2])
2020-10-19 11:57:24,815:INFO: GPU memory: 1697.0
2020-10-19 11:57:24,815:INFO: batch size 4 successful.
2020-10-19 11:57:24,817:INFO: GPU memory: 1697.0
2020-10-19 11:57:24,968:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 2])
2020-10-19 11:57:24,968:INFO: GPU memory: 1698.0
2020-10-19 11:57:24,969:INFO: batch size 8 successful.
2020-10-19 11:57:24,971:INFO: GPU memory: 1697.0
2020-10-19 11:57:25,262:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 2])
2020-10-19 11:57:25,263:INFO: GPU memory: 1698.0
2020-10-19 11:57:25,263:INFO: batch size 16 successful.
2020-10-19 11:57:25,265:INFO: GPU memory: 1697.0
2020-10-19 11:57:25,817:INFO: x shape = torch.Size([32, 512]), yhat.shape = torch.Size([32, 2])
2020-10-19 11:57:25,818:INFO: GPU memory: 1698.0
2020-10-19 11:57:25,818:INFO: batch size 32 successful.
2020-1

In [42]:
# BERT for pretraning

In [43]:
## Bert base

In [44]:
del gpu_model
torch.cuda.empty_cache()
check_memory()

2020-10-19 11:58:18,628:INFO: GPU memory: 1697.0


In [46]:
from transformers import BertForPreTraining
check_memory()
model = BertForPreTraining.from_pretrained('bert-base-uncased')
gpu_model = model.to(device)
check_memory()

2020-10-19 11:58:55,006:INFO: GPU memory: 1697.0
2020-10-19 11:58:55,577:INFO: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2020-10-19 11:58:55,579:INFO: Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2020-10-19 11:58:55,759:INFO: loading weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin from

In [47]:
run_max_batch_analysis(gpu_model)

2020-10-19 11:59:07,219:INFO: GPU memory: 2118.0
2020-10-19 11:59:07,257:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 512, 30522])
2020-10-19 11:59:07,258:INFO: GPU memory: 4691.0
2020-10-19 11:59:07,259:INFO: batch size 4 successful.
2020-10-19 11:59:07,261:INFO: GPU memory: 2118.0
2020-10-19 11:59:07,307:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 512, 30522])
2020-10-19 11:59:07,308:INFO: GPU memory: 7264.0
2020-10-19 11:59:07,309:INFO: batch size 8 successful.
2020-10-19 11:59:07,310:INFO: GPU memory: 2118.0
2020-10-19 11:59:07,470:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 512, 30522])
2020-10-19 11:59:07,471:INFO: GPU memory: 12410.0
2020-10-19 11:59:07,472:INFO: batch size 16 successful.
2020-10-19 11:59:07,474:INFO: GPU memory: 2118.0
2020-10-19 11:59:07,664:INFO: x shape = torch.Size([32, 512]), yhat.shape = torch.Size([32, 512, 30522])
2020-10-19 11:59:07,665:INFO: GPU memory: 22702.0
2020-10-19 11:59:07,666:

In [48]:
## Bert Large

In [51]:
del gpu_model
torch.cuda.empty_cache()
check_memory()

NameError: name 'gpu_model' is not defined

In [52]:
check_memory()

2020-10-19 12:00:07,219:INFO: GPU memory: 2118.0


In [53]:
model = BertForPreTraining.from_pretrained('bert-large-uncased')
check_memory()
gpu_model = model.to(device)
check_memory()

2020-10-19 12:00:11,821:INFO: loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-config.json from cache at /.cache/torch/transformers/6dfaed860471b03ab5b9acb6153bea82b6632fb9bbe514d3fff050fe1319ee6d.788fed32bb8481a9b15ce726d41c53d5d5066b04c667e34ce3a7a3826d1573d8
2020-10-19 12:00:11,823:INFO: Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

2020-10-19 12:00:11,901:INFO: loading weights file https://cdn.huggingface.co/bert-large-uncased-pytorch_model.bin from cache at /.cache/torch/transformers/73e65a464

In [54]:
run_max_batch_analysis(gpu_model)

2020-10-19 12:00:25,867:INFO: GPU memory: 3402.0
2020-10-19 12:00:25,942:INFO: x shape = torch.Size([4, 512]), yhat.shape = torch.Size([4, 512, 30522])
2020-10-19 12:00:25,943:INFO: GPU memory: 9825.0
2020-10-19 12:00:25,945:INFO: batch size 4 successful.
2020-10-19 12:00:25,947:INFO: GPU memory: 3402.0
2020-10-19 12:00:26,079:INFO: x shape = torch.Size([8, 512]), yhat.shape = torch.Size([8, 512, 30522])
2020-10-19 12:00:26,080:INFO: GPU memory: 16248.0
2020-10-19 12:00:26,081:INFO: batch size 8 successful.
2020-10-19 12:00:26,082:INFO: GPU memory: 3402.0
2020-10-19 12:00:26,392:INFO: x shape = torch.Size([16, 512]), yhat.shape = torch.Size([16, 512, 30522])
2020-10-19 12:00:26,393:INFO: GPU memory: 29095.0
2020-10-19 12:00:26,395:INFO: batch size 16 successful.
2020-10-19 12:00:26,396:INFO: GPU memory: 3402.0
exception <class 'RuntimeError'> : CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 47.46 GiB total capacity; 46.27 GiB already allocated; 155.00 MiB free; 46.40 GiB rese

In [ ]:
from transformers import ReformerForQuestionAnswering
from transformers import BertForQuestionAnswering
from transformers import LongformerForQuestionAnswering
from transformers import RobertaForQuestionAnswering
from transformers import XLMRobertaForQuestionAnswering
from transformers import TransfoXLPreTrainedModel